In [173]:
######### Importe librerias #########
import numpy as np
import math
import pandas as pd

In [174]:
######### Definicion valores #########
n = 5
tamaño_poblacion = 30
probabilidad_cruce = 0.2
probabilidad_mutacion = 0.02
numero_conexiones = int((n+1)*(n)/2)
posiciones = [np.array([0,0]) if i == 0 else np.random.randint(-30,30,2) for i in range(n+1)]
mejor_fitness = [np.zeros(numero_conexiones),1e10]
distancias = []
for i in range(0,len(posiciones)):
    for j in range(i,len(posiciones)):
        if(j != i):
            distancias.append(math.dist(posiciones[i],posiciones[j]))

In [175]:
######### Poblacion inicial #########
poblacion_inicial = np.random.randint(0,2,[tamaño_poblacion,numero_conexiones])

In [176]:
######### Funcion error #########
def funciones_penalizacion(genotipo,n):
    #genotipo = poblacion_inicial[0]
    #print(genotipo)
    penalizacion_central = False
    central = genotipo[:n]
    penalizacion_desconexion = False
    n_indice = n
    indice_inicial = 0
    indice_final = n_indice
    matriz_indices = np.zeros([n,n])
    for i in range(n):
        contador = 0
        valor = genotipo[indice_inicial:indice_final]
        if i == 0:
            central = valor
            #print(central)
            matriz_indices[n-1] = central
            if(sum(central) == 0):
                penalizacion_central = True
                break
        else:
            for j in valor:
                if j == 1:
                    matriz_indices[i-1][i-1] = 1
                    matriz_indices[i-1][i+contador] = 1
                contador += 1
        n_indice -= 1
        indice_inicial = indice_final
        indice_final = indice_final + n_indice
    busqueda_error = np.sum(matriz_indices, axis=0)
    #print(busqueda_error)
    if 0 in busqueda_error:
       penalizacion_desconexion = True
    if not penalizacion_central:
        None
    print(matriz_indices)
    #central = np.array(central)
    #print(penalizacion_central,penalizacion_desconexion)
    return penalizacion_central or penalizacion_desconexion, matriz_indices,central

    

In [223]:
vector_prueba = [0,0,0,1,0,1,0,0,1,1,0,1,0,1,0]
#vector_prueba = poblacion_inicial[0]
print(vector_prueba)
a,matriz_indices,central = funciones_penalizacion(vector_prueba,n)
if not a:
    conexion_central = np.array([1 if x==1 else 0 for x in central])
    indices_conexion = np.where(conexion_central==1)[0]
    print(f'primer: {indices_conexion}')
    contador_unos = len(indices_conexion)
    contador = 0
    while contador <= contador_unos:
        for i in indices_conexion:
            fila = np.array(matriz_indices[i])
            print(f'fila: {fila}')
            columna = np.array(matriz_indices[:,i].T)
            print(f'columna: {columna}')
            indices_fila = np.where(fila==1)[0]
            print(f'indices fila: {indices_fila}')
            indices_columna = np.where(columna[:-1]==1)[0]
            print(f'indices columna: {indices_columna}')
            print(f'conexion_central:{conexion_central}')
            for j in indices_fila:
                conexion_central[j]=1
            for k in indices_columna:
                conexion_central[k]=1
            print(f'conexion_central:{conexion_central}')
        indices_conexion = np.where(conexion_central==1)[0]
        print(f'indices conexion:{indices_conexion}')
        contador_unos = len(indices_conexion)
        print(f'contador unos:{contador_unos}')
        contador += 1
        

        
    print(conexion_central)
"""for i in indices_conexion:
    fila = np.array(matriz_indices[i])[0]
    print(fila)
    indices_fila = np.where(fila==1)[0]
    for j in range(1,len(indices_fila)):
        conexion_central[indices_fila[j]]=1"""



[0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0]
[[1. 1. 0. 0. 1.]
 [0. 1. 1. 0. 1.]
 [0. 0. 1. 0. 1.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]]
primer: [3]
fila: [0. 0. 0. 0. 0.]
columna: [0. 0. 0. 0. 1.]
indices fila: []
indices columna: []
conexion_central:[0 0 0 1 0]
conexion_central:[0 0 0 1 0]
indices conexion:[3]
contador unos:1
fila: [0. 0. 0. 0. 0.]
columna: [0. 0. 0. 0. 1.]
indices fila: []
indices columna: []
conexion_central:[0 0 0 1 0]
conexion_central:[0 0 0 1 0]
indices conexion:[3]
contador unos:1
[0 0 0 1 0]


'for i in indices_conexion:\n    fila = np.array(matriz_indices[i])[0]\n    print(fila)\n    indices_fila = np.where(fila==1)[0]\n    for j in range(1,len(indices_fila)):\n        conexion_central[indices_fila[j]]=1'

In [178]:
vector_penalizacion = []        
for i in poblacion_inicial: ## verifica conexiones
    print(i)
    vector_penalizacion.append(funciones_penalizacion(i))
penalizacion = [2000 if i else 1 for i in vector_penalizacion] ## Penaliza con un valor de 2000 si no se hacen todas las conexiones

[0 1 1 0 1 0 1 0 1 0 1 0 0 0 0]


TypeError: funciones_penalizacion() missing 1 required positional argument: 'n'

In [ ]:
def metricas(poblacion,distancias):
    fitness = np.dot(poblacion_inicial,distancias)  ## Calcula funcion fitness
    fitness = fitness*penalizacion
    fitness_generacion = fitness.sum()
    fitness_average = fitness/fitness_generacion
    suma_fitness_average = fitness_average.sum()
    maximo_fitness = 1-fitness_average
    ponderado = maximo_fitness*tamaño_poblacion
    tabla_promedio = pd.DataFrame({'fitness':fitness,'f/sum':fitness_average,'Max':maximo_fitness,'fi/f * tpobl':ponderado})
    print(tabla_promedio.head())
    return ponderado,fitness.min(),np.argwhere(fitness==fitness.min())


In [ ]:
ponderado_poblacion,fitness,posicion_fitness = metricas(poblacion_inicial,distancias)
if(fitness<mejor_fitness[1]):
    mejor_fitness = [poblacion_inicial[posicion_fitness[0][0]],fitness]

poblacion_intermedia = np.zeros([tamaño_poblacion,numero_conexiones])
for i in range(len(ponderado_poblacion)):
    indice_poblacion = np.argwhere(ponderado_poblacion==ponderado_poblacion.max())
    ponderado_poblacion[indice_poblacion] = ponderado_poblacion[indice_poblacion] - 1
    poblacion_intermedia[i]=(poblacion_inicial[indice_poblacion])

valor_cruce1 = np.random.randint(0,tamaño_poblacion)
cruce = np.random.randint(0,tamaño_poblacion)
while(cruce != valor_cruce1):
    cruce = np.random.randint(0,tamaño_poblacion)
valor_cruce2 = cruce
punto_cruce = np.random.randint(1,numero_conexiones)

print(poblacion_intermedia[valor_cruce1])
print(poblacion_intermedia[valor_cruce2])
a = poblacion_intermedia[valor_cruce1]
b = poblacion_intermedia[valor_cruce2]
poblacion_intermedia[valor_cruce1] = np.concatenate(a[:punto_cruce],b[punto_cruce:])
poblacion_intermedia[valor_cruce2] = np.concatenate(b[:punto_cruce],a[punto_cruce:])
print(poblacion_intermedia[valor_cruce1])
print(poblacion_intermedia[valor_cruce2])


         fitness     f/sum       Max  fi/f * tpobl
0  471963.308855  0.032664  0.967336     29.020086
1  657390.324486  0.045497  0.954503     28.635093
2  586009.553019  0.040557  0.959443     28.783298
3  376419.066821  0.026051  0.973949     29.218460
4  378590.812674  0.026202  0.973798     29.213951
[1. 1. 0. 1. 0. 0. 1. 1. 0. 1. 1. 0. 1. 0. 1.]
[1. 1. 0. 1. 0. 0. 1. 1. 0. 1. 1. 0. 1. 0. 1.]


TypeError: only integer scalar arrays can be converted to a scalar index

In [ ]:
#[0,0,1,0,0,1,0,0,0,0,1,1,0,1,1]
#[0,0,0,0,1,0,1,0,0,0,1,1,0,0,0]##

In [ ]:
busqueda_error = np.sum(matriz_indices, axis=0)
np.where(central==0)

NameError: name 'matriz_indices' is not defined

In [ ]:
fila2 = poblacion_inicial[0]
n2 = n
fila = fila2.copy()
fila = fila.tolist()
conexiones = np.zeros(n)
conexiones_centro = np.zeros(n)
print(fila)
centros = fila[:n]

centros_flag = False
nodoFlag = False
if (np.sum(centros)== 0):
    centros_flag = True
else:
    for i in range(len(centros)):
        if centros[i] == 1:
            conexiones_centro[i] = 1

del fila [0:n2]

if not centros_flag:
    for i in range(n2-1):
        n2 -=1 
        nodo = fila[0:n2]
        for j in range(len(nodo)):
            if nodo[j]==1:
                conexiones[i] = 1
                conexiones[j+i] = 1
        del fila [0:n2]
else:
    nodoFlag = True

print(conexiones)
print(conexiones_centro)
print(f'bandera 1: {centros_flag}, bandera 2: {nodoFlag}')

[1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0]
[1. 1. 1. 1. 0.]
[1. 1. 0. 1. 0.]
bandera 1: False, bandera 2: False
